# 🗽 Market Intelligence: Identifying High-Yield Airbnb Assets in NYC

**Objective**: Move beyond volume metrics to identify **efficiency** and **yield** in the NYC real estate market.

### 📊 Key Findings (The "Elite 6")
1.  **Efficiency Wins**: Financial District listings generate higher per-unit revenue than Midtown volume.
2.  **Asset Class**: Private Rooms offer a surprisingly cleaner efficiency ratio than Entire Homes.
3.  **Optimal Pricing**: The "Goldilocks Zone" for maximum yield is **$280 - $360**.
4.  **Risk Profile**: Even in Bear scenarios (P25), top quartiles remain profitable.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Setting Professional Theme
sns.set_theme(style="white", context="talk")

## 1. Data Loading
Data has been cleaned and enriched via our SQL ETL pipeline (see `sql/` folder).
Key metrics added: `RevPAR`, `Occupancy Rate`, `Annual Revenue`.

In [ ]:
df = pd.read_csv("data/comprehensive_data.csv")
print(f"Loaded {df.shape[0]} listings.")
df.head(3)

## 2. Market Matrix: Identifying "Cash Cows"
**Question**: Should we invest in high-volume neighborhoods or high-efficiency ones?

In [ ]:
# Aggregation
nbhd_stats = df.groupby('neighbourhood').agg({
    'annual_revenue': 'sum',
    'id': 'count'
}).reset_index()
nbhd_stats['revenue_per_listing'] = nbhd_stats['annual_revenue'] / nbhd_stats['id']

# Filter sparse neighborhoods
nbhd_stats = nbhd_stats[nbhd_stats['id'] > 50]

plt.figure(figsize=(10, 8))
p = sns.scatterplot(
    data=nbhd_stats, 
    x='annual_revenue', y='revenue_per_listing', 
    size='id', sizes=(50, 500), alpha=0.7, 
    hue='revenue_per_listing', palette='viridis', legend=False
)

# Annotate Leaders
top_efficiency = nbhd_stats.sort_values('revenue_per_listing', ascending=False).head(5)
for i, row in top_efficiency.iterrows():
     p.text(row.annual_revenue, row.revenue_per_listing+1000, row.neighbourhood, 
            ha='center', size='small', weight='semibold')

plt.title("1. Market Matrix: Identifying 'Cash Cows'", fontweight='bold')
plt.xlabel("Total Market Revenue (Volume)")
plt.ylabel("Avg Revenue Per Listing (Efficiency)")
plt.grid(True, linestyle=':', alpha=0.5)
plt.show()

## 3. Asset Efficiency: RevPAR Comparison
**Question**: Does the higher price of Entire Homes justify the cost?

In [ ]:
plt.figure(figsize=(10, 6))
clean_df = df[df['room_type'] != 'Hotel room']
efficiency = clean_df.groupby('room_type').agg({'revpar': 'mean', 'id': 'count'}).reset_index()

ax = sns.barplot(data=efficiency, x='room_type', y='revpar', palette="Blues_d")

# Annotate
for i, row in efficiency.iterrows():
    ax.text(i, row['revpar'] + 2, f"${row['revpar']:.0f}\n(n={row['id']})", 
            ha='center', va='bottom', fontweight='bold')

plt.title("2. Asset Efficiency: RevPAR Comparison", fontweight='bold')
plt.ylabel("")
plt.yticks([])
plt.xlabel("")
sns.despine(left=True)
plt.show()

## 4. Pricing Strategy: The "Optimal Zone"
**Question**: At what price point does occupancy collapse?

In [ ]:
plt.figure(figsize=(10, 8))
viz_df = df[(df['nightly_rate'] < 500) & (df['occupancy_rate'] > 0)].copy()
p90 = viz_df['revpar'].quantile(0.90)
viz_df['is_top'] = viz_df['revpar'] > p90

# Density Plot (Low Alpha)
sns.scatterplot(data=viz_df[~viz_df['is_top']], x='nightly_rate', y='occupancy_rate', 
                color='#95a5a6', alpha=0.1, s=40, label='Standard')
sns.scatterplot(data=viz_df[viz_df['is_top']], x='nightly_rate', y='occupancy_rate', 
                color='#2ecc71', alpha=0.8, s=60, label='Top 10% Yield')

plt.axvline(viz_df[viz_df['is_top']]['nightly_rate'].median(), color='#27ae60', linestyle='--')
plt.title("3. Pricing Strategy: Efficiency Frontier", fontweight='bold')
plt.legend(loc='upper right')
sns.despine()
plt.show()

## 5. Risk Analysis: Historical Scenarios
**Question**: What are the realistic Bear/Bull outcomes?

In [ ]:
p25 = df['annual_revenue'].quantile(0.25)
p50 = df['annual_revenue'].median()
p75 = df['annual_revenue'].quantile(0.75)

scenarios = pd.DataFrame({
    'Scenario': ['Bear (P25)', 'Base (Median)', 'Bull (P75)'],
    'Revenue': [p25, p50, p75]
})

plt.figure(figsize=(10, 6))
ax = sns.barplot(data=scenarios, x='Scenario', y='Revenue', palette=["#7f8c8d", "#2980b9", "#27ae60"])

for i, v in enumerate(scenarios['Revenue']):
    ax.text(i, v + 500, f"${v:,.0f}", ha='center', va='bottom', fontweight='bold')

plt.title("4. Investment Benchmarks: Illustrative Scenarios", fontweight='bold')
plt.ylabel("")
plt.yticks([])
sns.despine(left=True)
plt.show()